In [1]:
%matplotlib inline

from IPython import display
from data_generator import generate_data

import numpy as np
import matplotlib.pyplot as plt

from hydroml import model

import torch
import torch.nn as nn

In [2]:
# Decide which device we want to run on
ngpu = 0
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")